In [134]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
from pathlib import Path

In [135]:
data_dir = Path("/Users/alxli/Documents/CS Projects/Datasets/asl_dataset")

In [136]:
batch_size = 32
img_height = 400
img_width = 400

In [137]:
image_count = (len(list(data_dir.glob('*/*.jpeg'))))
print(image_count)

2515


In [138]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=69,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2515 files belonging to 36 classes.
Using 2012 files for training.


In [139]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=69,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2515 files belonging to 36 classes.
Using 503 files for validation.


In [140]:
class_names = train_ds.class_names
print(len(class_names))

36


In [141]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 400, 400, 3)
(32,)


In [142]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

In [143]:
# speeds up preprocessing

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [144]:
num_classes = 36

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [145]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [146]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=4
)

Epoch 1/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 60s 942ms/step - accuracy: 0.3512 - loss: 2.6087 - val_accuracy: 0.8748 - val_loss: 0.4573
Epoch 2/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 56s 881ms/step - accuracy: 0.9293 - loss: 0.2216 - val_accuracy: 0.9205 - val_loss: 0.3015
Epoch 3/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 57s 905ms/step - accuracy: 0.9819 - loss: 0.0548 - val_accuracy: 0.9344 - val_loss: 0.2823
Epoch 4/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 55s 873ms/step - accuracy: 0.9918 - loss: 0.0317 - val_accuracy: 0.9483 - val_loss: 0.1913
